In [2]:
 !pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.9 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
print(tf.__version__)

2.15.0


In [5]:
fashion_mnist=keras.datasets.fashion_mnist

In [6]:
(train_images,train_labels), (test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [7]:
train_images=train_images/255.0
test_images=test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-11-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [12]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist Fashion')

In [13]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 24s]
val_accuracy: 0.8708333373069763

Best val_accuracy So Far: 0.9039999842643738
Total elapsed time: 00h 03m 15s


In [14]:
model=tuner_search.get_best_models(num_models=1)[0]

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 64)        51264     
                                                                 
 flatten (Flatten)           (None, 25600)             0         
                                                                 
 dense (Dense)               (None, 48)                1228848   
                                                                 
 dense_1 (Dense)             (None, 10)                490       
                                                                 
Total params: 1281434 (4.89 MB)
Trainable params: 1281434 (4.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1719 - accuracy: 0.9352 - val_loss: 0.2502 - val_accuracy: 0.9108
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1372 - accuracy: 0.9479 - val_loss: 0.2774 - val_accuracy: 0.9085
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1081 - accuracy: 0.9596 - val_loss: 0.2979 - val_accuracy: 0.9087
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0835 - accuracy: 0.9690 - val_loss: 0.3559 - val_accuracy: 0.9110
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0701 - accuracy: 0.9733 - val_loss: 0.3780 - val_accuracy: 0.9098
Epoch 9/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0564 - accuracy: 0.9792 - val_loss: 0.3964 - val_accuracy: 0.9112
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0444 - accuracy: 0.9831 - val_loss: 0.4638 - val_accuracy